# Libraries and data

# Useful functions

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as st
import statsmodels.stats.weightstats as sm

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 2-Tailed tests with known variance

You have invested thousands of dollars per employee to improve their satisfaction and productivity. Your goal is to improve from the average of 54 cars produced so far, with a corresponding standard deviation (of the population) of 2.
Bruno believes the opposite. That the benefits and other factors like the constant raining are hurting production due to constant sickness. The agreed confidence level between you both is 95%

- Null Hypythesis: Average is 54
- Alternate Hypothesis: Average is not 54

In [2]:
data_loc = "../../Z_Resources/Statistics with Python/Inferential Statistics/Hypothesis Testing/"

In [3]:
df_main = pd.read_csv(data_loc+"tesla_main.csv")
df_main.head()

,Production Date,Defects Found,Cars Produced,Weather Condition,Workers on Shift
0,2023-01-01,3,55,Rainy,20
1,2023-01-02,2,57,Rainy,19
2,2023-01-03,1,54,Rainy,21
3,2023-01-04,0,56,Rainy,22
4,2023-01-05,2,59,Rainy,20


In [4]:
#population
mean_pop = 54
sd_pop = 2
confidence = .95
alpha = 1- confidence
#sample
mean_sample = df_main["Cars Produced"].mean()
sample_size = df_main["Cars Produced"].count()


In [5]:
z_score = (mean_sample-mean_pop)/(sd_pop/np.sqrt(sample_size))
z_score

4.112619161025777

In [6]:
# calculate p-value for the z_score (two tails)
tails = 2
p_value = st.norm.sf(abs(z_score))*tails
p_value

3.9119543361101206e-05

In [7]:
#Interpret the p_value
def p_value_reader(p_value, alpha):
    if p_value < alpha:
        print("Reject the Null Hypothesis")
    else:
        print("Fail to reject the Null Hypothesis")

In [8]:
# Z-test Function
def z_test(df: pd.Series, pop_mean: float, confidence:float, tails: int = 2,pop_std = None) -> None:
    sample_mean = df.mean()
    sample_size = df.count()
    alpha = 1-confidence
    z_score = (sample_mean-pop_mean)/(pop_std/np.sqrt(sample_size))
    p_value = st.norm.sf(abs(z_score))*tails
    p_value_reader(p_value, alpha)

In [9]:
z_test(df_main["Cars Produced"], 54,.95, 2,2)

Reject the Null Hypothesis


# 2-Tailed tests with unknown variance

Social Media has been all over Tesla. The engines from a couple of cars started to catch smoke. Even worse, the cars were from high profile customers. You talk to your employees who tell you that the number of defects is within normal average of 2.2. Bruno asked you to investigate the situation yourself. Since the car production has suffered many changes in the past few months, there is no data about the population.

In [10]:
pop_mean = 2.2
data = df_main["Defects Found"]
sample_size = data.count()
sample_mean = data.mean()
confidence = .95
alpha = 1-confidence
sample_sd = data.std()

In [11]:
# t-score
t_score = (sample_mean-pop_mean)/(sample_sd/np.sqrt(sample_size))
t_score

1.1259778359082033

In [12]:
# p_value
tails = 2
p_value = st.t.sf(abs(t_score),df = sample_size - 1)*tails
p_value

0.26515424936297255

In [13]:
p_value_reader(p_value,alpha)

Fail to reject the Null Hypothesis


- average no of defects are 2.2

In [14]:
st.ttest_1samp(a=data, alternative="two-sided", popmean=pop_mean)

Ttest_1sampResult(statistic=1.1259778359082033, pvalue=0.26515424936297255)

# 2-Tailed Paired T-Test

The Sales department has been very critical of Tesla recently, saying that they have been getting many complains from customers that claim that the car is taking longer than expected. They have even voiced their concerns to Bruno!

Your department, on the other hand, says that the production has been stable over time and going according to plan. They fire back and say that the sales department has been selling too much. You decide to take initiative to see if the production is stable in the last 2 months.







In [15]:
df_paired = pd.read_csv(data_loc+"tesla_paired.csv")
df_paired.head()

,Day,Month 1,Month 2
0,1,58,54
1,2,54,56
2,3,57,55
3,4,55,53
4,5,55,52


In [16]:
len(df_paired)

30

In [17]:
x1 = df_paired['Month 1'].mean()
x2 = df_paired['Month 2'].mean()
SD = (df_paired['Month 1'] - df_paired['Month 2']).std()
n = len(df_paired)

In [18]:
dof = n-1
tails = 2
confidence = .95
alpha = 1-confidence

In [19]:
t_score = (x2-x1)/(SD/(n**.5))
t_score 

-3.170873895434035

In [20]:
p_value = st.t.sf(abs(t_score), df = dof)*tails
p_value

0.003574334255295166

In [21]:
p_value_reader(p_value,alpha)

Reject the Null Hypothesis


In [22]:
t_score,p_value = st.ttest_rel(df_paired['Month 1'],
                              df_paired['Month 2'],
                              alternative='two-sided')
p_value_reader(p_value,alpha)

Reject the Null Hypothesis


# 2-Tailed Two Sample T-Test

The Tesla factory you manage has two shifts during the day. You are present during shift 2, but not shift 1. Your second in command who you that shift 2 is doing great.

Of course, you are aware that, to prove to Bruno that both shifts work with similar productivity, you need to show them numbers. Of course, nothing is better than hypothesis testing

In [23]:
df_2sample = pd.read_csv(data_loc+"tesla_2sample.csv")
g1 = df_2sample['Shift 1'].dropna()
g2 = df_2sample['Shift 2'].dropna()
df_2sample.head()

,Day,Shift 1,Shift 2
0,1,53,49.0
1,2,61,57.0
2,3,72,68.0
3,4,59,47.0
4,5,62,60.0


In [24]:
stats,p_value=  st.levene(g1,g2)
p_value

0.044682721966871876

In [25]:
p_value_reader(p_value,0.05)

Reject the Null Hypothesis


In [26]:
# There fore we use Welch's test
st.ttest_ind(
    g1,g2,equal_var=False,
    alternative='two-sided'
)

Ttest_indResult(statistic=3.0606654074470834, pvalue=0.0034724013986656174)

# 1-Tailed Tests with Known Variance

You have invested thousands of dollars per employee to improve their satisfaction and productivity. Your goal is to improve from the average of 54.5 cars produced so far, with a corresponding standard deviation (of the population) of 2.

Brun does not believe it and asks for proof. Statistical proof of course :)

- Null Hypothesis: No improvement
- Alternative Hypothesis: Smple mean is greater than pop mean(54.5)

In [27]:
#Data
pop_mean = 54.5
sd_pop = 2
confidence = 0.95
alpha = 1- confidence
mean_sample= df_main['Cars Produced'].mean()
sample_size = df_main["Cars Produced"].count()
mean_sample,sample_size

(55.10909090909091, 55)

In [ ]:

z_score = (mean_sample-pop_mean)/(sd_pop/np.sqrt(sample_size))
p_value = st.norm.sf(abs(z_score))*1
p_value_reader(p_value, alpha)
z_score,p_value

# 1-Tailed Test with Unknown Variance

Social Media has been all over Tesla. They say that more and more people are complaining about defects. They claim that improvements are urgently needed. You talk to your employees who tell you that the number of defects is within normal average of 2.4, maybe even better than that.

You decide to investigate the situation yourself. Since the car production has suffered many changes in the past few months, there is no data about the population.

# 1-Tailed Paired T-Test

The Sales department has been very critical of Tesla recently, saying that they have been getting many complains from customers that claim that the car is taking longer than expected and that the production slowed down last month. They have even voiced their concerns to Bruno.

Your department, on the other hand, says that the production is at least as good as before. The historical pattern is that productivity increases over time

# 1-Tailed Two-Sample T-Test

The Tesla factory you manage has two shifts during the day. You are present during shift 2, but not shift 1. You have your second in command who tells you that shift 1 is doing great. In fact, her recent incentives led to higher efficiency during the second shift. Let's if that is true.

# Chisquare Test

You are a car production manager in charge of two factories, Factory A and Factory B. Both factories have recently implemented different quality control measures to reduce the number of defective cars produced in each of the three car categories: sedan, SUV, and truck. You want to determine if the quality control measures have had any significant impact on reducing the proportion of defective cars across the three categories.